# 3 Data Preparation

---

In this phase, I will perform the following tasks:

1. Handling outliers
2. Scale numerical features
3. Enumeration of categorical variables (using <em>one hot encoding</em>)
4. Feature analysis
5. Data Resampling
6. Conclusion



**Setup**

Lead the necessary libraries and output data file from the previous step.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)

avocados_df = pd.read_csv("../data/interim/avocado_cleaned.csv")
avocados_df.head()

### 3.1 Handling Outliers

---

#### Average Price

There are two commonly used methods to identify outliers: **Z-score** and **IQR**. Median and IQR measures the central tendency and spread, respectively, but are robust against outliers and non-normal data [2]. As the dataset is small and there have been so many potential outliers highlighted in the box-plots in the previous steps, the **Z-score** might flag too many points as outliers. Therefore, I will use the **IQR** method to identify outliers.

In [ ]:
average_price_Q1 = avocados_df['average_price'].quantile(0.25)
average_price_Q3 = avocados_df['average_price'].quantile(0.75)
average_price_IQR = average_price_Q3 - average_price_Q1

average_price_lower_bound = average_price_Q1 - 1.5 * average_price_IQR
average_price_upper_bound = average_price_Q3 + 1.5 * average_price_IQR

average_price_outliers_mask = (avocados_df['average_price'] < average_price_lower_bound) | (avocados_df['average_price'] > average_price_upper_bound)
average_price_outliers = avocados_df[average_price_outliers_mask]

plt.figure(figsize=(10, 6))
plt.boxplot(avocados_df['average_price'])
plt.title('Boxplot of Average Price')
plt.ylabel('Average Price')
plt.show()

print("Q0.25:", average_price_Q1)
print("Q0.75:", average_price_Q3)
print('IQR:', average_price_IQR)
print("Lower bound:", average_price_lower_bound)
print("Upper bound:", average_price_upper_bound)
print("Number of average_price:", avocados_df['average_price'].shape[0])
print("Number of outliers:", average_price_outliers.shape[0])
avocados_df[['average_price']].describe()

Since the `average_price` and `total_volume` are the target variables, and the number of outliers is not significant, I will not remove any outliers from these columns.

#### PLU 4046

As demonstrated in the previous sections by the boxplots, the PLU code columns contain many outliers that need to be investigated and addressed. I will use the **IQR** method to identify the outliers. These outliers are likely to be the result of the ambigious regions and the aggregation of those regions.

First, I will calculate the IQR for the PLU 4046 column, describe the outliers and plot the boxplot.

In [ ]:
plu_4046_Q1 = avocados_df['4046'].quantile(0.25)
plu_4046_Q3 = avocados_df['4046'].quantile(0.75)
plu_4046_IQR = plu_4046_Q3 - plu_4046_Q1

old_plu_4046_lower_bound = plu_4046_Q1 - 1.5 * plu_4046_IQR # the result of this calculation is negative, as price cannot be negative, we will use 0 as the lower bound
plu_4046_lower_bound = 0
plu_4046_upper_bound = plu_4046_Q3 + 1.5 * plu_4046_IQR

plu_4046_outliers_mask = (avocados_df['4046'] < plu_4046_lower_bound) | (avocados_df['4046'] > plu_4046_upper_bound)
plu_4046_outliers = avocados_df[plu_4046_outliers_mask]

plt.figure(figsize=(10, 6))
plt.boxplot(avocados_df['4046'])
plt.title('Boxplot of PLU 4046 Volume Sale')
plt.ylabel('PLU 4046 Volume Sale')
plt.show()

print("Q0.25:", plu_4046_Q1)
print("Q0.75:", plu_4046_Q3)
print('IQR:', plu_4046_IQR)
print("Lower bound (Actual):", old_plu_4046_lower_bound)
print("Lower bound (Manual):", plu_4046_lower_bound)
print("Upper bound:", plu_4046_upper_bound)
print("Number of PLU 4046:", avocados_df['4046'].shape[0])
print("Number of outliers for PLU 4046:", plu_4046_outliers.shape[0])

avocados_df['4046'].describe().apply(lambda x: f'{x:,.2f}')

The identified outliers in the PLU 4046 are numerous; hence, I need to investigate whether they are the result of an error or if they are valid data points.

> **Note**: The lower bound being negative could either due to negative values or skewed distribution of extreme outliers, affecting the IQR calculation. Inorder to avoid removing any valid data points, I will use 0 as the lower bound when ever it is calculated to be negative.

> **Note**: Since the regions of the dataset were scrambled, these outliers might simply be the result of aggregation of different regions of different sizes. Hence the data is better to be normalized by using the mode of <em>each month</em> instead of the whole data or each year, in order not to lose the monthly seasonality of the data.

Next, I will plot the volume sale of PLU 4046 for each month, with and without the outliers and also with the outliers replaced by the mode of each month using boxplots.

In [ ]:
plt.figure(figsize=(25, 10))
plt.subplot(1, 3, 1)
sns.boxplot(x='month_name', y='4046', data=avocados_df)
plt.title('PLU 4046 Volume Sale Boxplot for Each Month')
plt.ylabel('PLU 4046 Volume Sale')
plt.xticks(rotation=45)
plt.xlabel('Month')

plt.subplot(1, 3, 2)
zonder_outliers = avocados_df.drop(plu_4046_outliers.index)
sns.boxplot(x='month_name', y='4046', data=zonder_outliers)
plt.title('PLU 4046 Volume Sale Boxplot (Without Outliers) for Each Month')
plt.ylabel('PLU 4046 Volume Sale')
plt.xticks(rotation=45)
plt.xlabel('Month')

plt.subplot(1, 3, 3)
mode_season_4046 = avocados_df.copy()
mode_season_4046['4046_replaced_outliers'] = mode_season_4046['4046']
for month in mode_season_4046['month_name'].unique():
    mask = (mode_season_4046['month_name'] == month) & plu_4046_outliers_mask
    mode = mode_season_4046[mask]['4046'].mode()
    mode_season_4046.loc[mask, '4046_replaced_outliers'] = mode[0]

sns.boxplot(x='month_name', y='4046_replaced_outliers', data=mode_season_4046)
plt.title('PLU 4046 Volume Sale Boxplot (Outliers Replaced by Mode) for Each Month')
plt.ylabel('PLU 4046 Volume Sale')
plt.xticks(rotation=45)
plt.xlabel('Month')

As the analysis above shows the importance of not removing the outliers but to normalize the data by using the mode of each month and make sure the seasonality of the data is kept intact. As the extreme outliers (big regions) are most likely an aggregation of smaller regions. I tried to replace the outliers of each month with the mode value of it. The result seems to be more balanced and the seasonality is preserved, although shifted as evident that for example 'September's maximum value was higher than 'Augusts' maximum value, but after replacing the outliers with the mode value, it is the other way around. However, it could possibly be because of the regions aggregations.

I will apply the changes to the main dataframe instance:

In [ ]:
avocados_df['4046'] = mode_season_4046['4046_replaced_outliers']
avocados_df['4046'].describe().apply(lambda x: f'{x:,.2f}')

#### PLU 4225

Now I will do the same to the other PLU codes

In [ ]:
plu_4225_Q1 = avocados_df['4225'].quantile(0.25)
plu_4225_Q3 = avocados_df['4225'].quantile(0.75)
plu_4225_IQR = plu_4225_Q3 - plu_4225_Q1

old_plu_4225_lower_bound = plu_4225_Q1 - 1.5 * plu_4225_IQR # the result of this calculation is negative, as price cannot be negative, we will use 0 as the lower bound
plu_4225_lower_bound = 0
plu_4225_upper_bound = plu_4225_Q3 + 1.5 * plu_4225_IQR

plu_4225_outliers_mask = (avocados_df['4225'] < plu_4225_lower_bound) | (avocados_df['4225'] > plu_4225_upper_bound)
plu_4225_outliers = avocados_df[plu_4225_outliers_mask]

print("Q0.25:", plu_4225_Q1)
print("Q0.75:", plu_4225_Q3)
print('IQR:', plu_4225_IQR)
print("Lower bound (Actual):", old_plu_4225_lower_bound)
print("Lower bound (Manual):", plu_4225_lower_bound)
print("Upper bound:", plu_4225_upper_bound)
print("Number of PLU 4225:", avocados_df['4225'].shape[0])
print("Number of outliers for PLU 4225:", plu_4225_outliers.shape[0])

avocados_df['4225'].describe().apply(lambda x: f'{x:,.2f}')

In [ ]:
mode_season_4225 = avocados_df.copy()
mode_season_4225['4225_replaced_outliers'] = mode_season_4225['4225']
for month in mode_season_4225['month_name'].unique():
    mask = (mode_season_4225['month_name'] == month) & plu_4225_outliers_mask
    mode = mode_season_4225[mask]['4225'].mode()
    mode_season_4225.loc[mask, '4225_replaced_outliers'] = mode[0]

avocados_df['4225'] = mode_season_4225['4225_replaced_outliers']
avocados_df['4225'].describe().apply(lambda x: f'{x:,.2f}')

#### PLU 4770

In [ ]:
plu_4770_Q1 = avocados_df['4770'].quantile(0.25)
plu_4770_Q3 = avocados_df['4770'].quantile(0.75)
plu_4770_IQR = plu_4770_Q3 - plu_4770_Q1

old_plu_4770_lower_bound = plu_4770_Q1 - 1.5 * plu_4770_IQR  # the result of this calculation is negative, as price cannot be negative, we will use 0 as the lower bound
plu_4770_lower_bound = 0
plu_4770_upper_bound = plu_4770_Q3 + 1.5 * plu_4770_IQR

plu_4770_outliers_mask = (avocados_df['4770'] < plu_4770_lower_bound) | (avocados_df['4770'] > plu_4770_upper_bound)
plu_4770_outliers = avocados_df[plu_4770_outliers_mask]

print("Q0.25:", plu_4770_Q1)
print("Q0.75:", plu_4770_Q3)
print('IQR:', plu_4770_IQR)
print("Lower bound (Actual):", old_plu_4770_lower_bound)
print("Lower bound (Manual):", plu_4770_lower_bound)
print("Upper bound:", plu_4770_upper_bound)
print("Number of PLU 4770:", avocados_df['4770'].shape[0])
print("Number of outliers for PLU 4770:", plu_4770_outliers.shape[0])

avocados_df['4770'].describe().apply(lambda x: f'{x:,.2f}')


### 
Convert Categorical values to numerical values for modeling.

In [ ]:
mode_season_4770 = avocados_df.copy()
mode_season_4770['4770_replaced_outliers'] = mode_season_4770['4770']
for month in mode_season_4770['month_name'].unique():
    mask = (mode_season_4770['month_name'] == month) & plu_4770_outliers_mask
    mode = mode_season_4770[mask]['4770'].mode()
    mode_season_4770.loc[mask, '4770_replaced_outliers'] = mode[0]

avocados_df['4770'] = mode_season_4770['4770_replaced_outliers']
avocados_df['4770'].describe().apply(lambda x: f'{x:,.2f}')

Now I will plot each PLU code again using boxplots to see the changes.

In [ ]:
plt.figure(figsize=(15, 6))
plt.subplot(1, 3, 1)
plt.title('4225')
plt.boxplot(avocados_df['4225'])
plt.subplot(1, 3, 2)
plt.title('4046')
plt.boxplot(avocados_df['4046'])
plt.subplot(1, 3, 3)
plt.boxplot(avocados_df['4770'])
plt.title('4770')
plt.ylabel('PLU 4225 Volume Sale')

Afte the imputation of outliers by the mode of each month, the boxplots show that all the 3 PLU codes are now more balanced and normalized, containing minimum outliers.

### 3.2 Scale Numerical Features

---

In order to make it easier to work with data (as this dataset has many large scale features) and also to improve the performance of the model, I will scale the numerical features in this step. This can be achieved using techniques such as Min-Max and Max-Abs Scaling or Standardization. These methods are generally sensitive to outliers, however, Z-Score Standardization is said to be robust against outliers [3]. Even though I have handled the outliers as much as I could, I still do not trust the data, so I will use the Z-Score Standardization method.

In [ ]:
scaler = StandardScaler()
numerical_columns = ['average_price', 'total_volume', '4046', '4225', '4770', 'total_bags']
avocados_df[numerical_columns] = scaler.fit_transform(avocados_df[numerical_columns])
avocados_df[numerical_columns].head()

### 3.3 Enumeration of Categorical Variables (One Hot Encoding)

---

In order to use the categorical variables in the model, I will convert them to numerical values using the **One Hot Encoding** method. This method has been adopted for its simplicity and effectiveness. The following columns will be converted:

- `type`
- `area`
- `season`

In <em>One Hot Encoding</em>, each category is represented as a binary vector. The vector is all zero (or false) values except for the index of the category, which is marked with a 1 (or true).

In [ ]:
avocados_df_copy = avocados_df.copy()

avocados_df = pd.get_dummies(avocados_df, columns=['area', 'type', 'month_name'], drop_first=False, dtype=int)

print(avocados_df.shape)

avocados_df.columns = avocados_df.columns.str.lower().str.replace(' ', '_')

avocados_df.head()

Validate the enumeration:

In [ ]:
area_cols = [col for col in avocados_df.columns if col.startswith('area_')]
type_cols = [col for col in avocados_df.columns if col.startswith('type_')]
month_name_cols = [col for col in avocados_df.columns if col.startswith('month_name_')]

print("Validating one-hot encoding:")
print("-" * 50)

area_sums = avocados_df[area_cols].sum(axis=1)
print("\nArea encoding validation:")
print(f"Number of unique areas per row: {area_sums.unique()}")
print(f"All rows have exactly one area: {(area_sums == 1).all()}")
print(f"Areas encoded: {', '.join(col.replace('area_','') for col in area_cols)}")

type_sums = avocados_df[type_cols].sum(axis=1)
print("\nType encoding validation:")
print(f"Number of unique types per row: {type_sums.unique()}")
print(f"All rows have exactly one type: {(type_sums == 1).all()}")
print(f"Types encoded: {', '.join(col.replace('type_','') for col in type_cols)}")

month_sums = avocados_df[month_name_cols].sum(axis=1)
print("\nMonth encoding validation:")
print(f"Number of unique months per row: {month_sums.unique()}")
print(f"All rows have exactly one month: {(month_sums == 1).all()}")
print(f"Months encoded: {', '.join(col.replace('month_name_','') for col in month_name_cols)}")

print("\nChecking for null values after encoding:")
null_counts = avocados_df[area_cols + type_cols + month_name_cols].isnull().sum()
print(f"Total null values: {null_counts.sum()}")
if null_counts.sum() > 0:
    print("Columns with null values:")
    print(null_counts[null_counts > 0])

### 3.4 Feature Analysis

---

#### Heatmap

I will try to identify the correlation between the features again after the recent changes to see if my modifications have had any effect. This way I can better select my target features for the model.

**Entire Dataset Correlation**

In [ ]:
avocados_df['date'] = pd.to_datetime(avocados_df['date'])

corr_matrix = avocados_df.corr()

plt.figure(figsize=(20, 14))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title("Correlation Matrix of Avocado Dataset")
plt.show()

Unfortunately, I do not see much difference in the heatmap. The correlation between the features and the target variables is still very low. The only new observation is that the `type_organic` column has a positive correlation with the target variable `average_price`!

Based on the heatmap, I can deduct the following key points:

- The `conventional` avocado type has a strong <em>positive</em> correlation with the PLU codes.
- The `conventional` avocado type has a <em>positive</em> correlation with the `total_bags` and `total_volume`.
- The `conventional` avocado type has a <em>negative</em> correlation with the `average_price`.
- The `conventional` avocado type has a strong <em>negative</em> correlation with the PLU codes.
- The `conventional` avocado type has a <em>negative</em> correlation with the `total_bags` and `total_volume`.
- The `conventional` avocado type has a <em>positive</em> correlation with the `average_price`.
- The new Area columns are slightly effective on the `average_price` and PLU codes, but the effect is not very strong. Hence, I will remove them.
- The new Months columns are not very effective on the target, either my data preparation has not been done correctly or the data is not suitable for this kind of analysis. As they do not have much effect, I will remove them.
- The `total_bags` and `total_volume` are highly correlated with each other, which is expected as they are both measures of the same thing (this could suggest that one of them is redundant, so I will remove `total_bags`).
- The `year` column also does not seem to have any effect on any other feature. I will remove this column too.
- The `total_bags` and `total_volume` are negatively correlated with the `average_price`.

In general, I believe that the modifications I have made have made the data more balanced and the correlation between the features and the target variables is now more visible. Hence, I will proceed with the next step.

Remove the features do not have any string or meaningful correlations:

In [ ]:
columns_to_drop = [col for col in avocados_df.columns if col.startswith(('area_', 'month_name_'))]
avocados_df = avocados_df.drop(columns=columns_to_drop)

avocados_df = avocados_df.drop(columns=['total_bags', 'year'])
avocados_df.head()


### 3.5 Data Resampling

---

Resampling the data to match the aimed prediction frequency (monthly) can help the forecasting model capture important monthly trends and simplify the modeling process [1]. In order to do so, I will resample the data to monthly frequency and plot the `Average Price` and `Total Volume` trends.

In [ ]:
avocados_df["date"] = pd.to_datetime(avocados_df["date"])

avocados_df = avocados_df.groupby(avocados_df["date"].dt.to_period("M")).mean(numeric_only=True)
avocados_df = avocados_df.reset_index()
avocados_df["date"] = avocados_df["date"].dt.to_timestamp()
monthly_df = avocados_df.set_index("date")

plt.figure(figsize=(15,8))
plt.plot(avocados_df.index, avocados_df["average_price"], label="Average Price")
plt.plot(avocados_df.index, avocados_df["total_volume"], label="Total Volume")
plt.title("Monthly Avocado Trends")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Now the data is resampled to monthly frequency and seems to be ready for the next step (Modeling). Even though the data has majorly lost its granularity, resampling is an important process to help the forecasting model capture the monthly trends.

**Save Output**

Save the output to a new CSV file for the next step (3 Modeling).

In [17]:
avocados_df = avocados_df.sort_index(axis=1)
avocados_df.to_csv("../data/processed/avocado_processed.csv", index=False)

### 3.6 Conclusion

---

So far I have prepared the data as much as I could and have made the following changes:

- Identified the `average_price` outliers using the **IQR** method but did not delete them.
- Handled the `PLU 4046`, `PLU 4225`, and `PLU 4770` outliers using the **IQR** method by replacing them with the mode of each month.
- Enumerated the categorical variables using the **One Hot Encoding** method.
- Analyzed the correlation between the features and the target variables after the recent changes.

### 3.7 References

[1] N. Kourentzes, F. Petropoulos, and J. R. Trapero, “Improving forecasting by estimating time series structural components across multiple frequencies,” International Journal of Forecasting, vol. 30, no. 2, pp. 291–302, Apr. 2014. doi: 10.1016/j.ijforecast.2013.09.006.

[2] A. K. Mishra, “Outlier Detection in Time-Series Receive Signal Strength Observation Using Z-Score Method with Sn Scale Estimator for Indoor Localization,” IEEE Sensors Journal, vol. 19, no. 13, pp. 5148–5155, July 2019. doi: 10.1109/JSEN.2019.2909738.

[3] A. Yıldız, M. E. Er, A. Bursalı, T. Çolakoğlu, and E. C. Erkuş, “The Effects of Data Standardization and Normalization Techniques in Click Through Rate Prediction,” in Proceedings of the 11th International Conference on Advanced Technologies (ICAT’23), 2023, pp. 200–203. doi: 10.58190/icat.2023.48.

######
The density of your data should be the same as how much you want to predict. If you want to predict the next 5 mins, your data density must be 5 mins.

First, define prediction rate (how long).